In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel
from langchain_core.tracers.context import collect_runs
from langchain_core.runnables.graph_ascii import draw_ascii 
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("OPENAI_ENDPOINT")
model_name = "gpt-4o-mini"
temp=0.0

llm = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model=model_name,
    temperature=temp
)

## **Chaining invocations**

In [2]:
prompt = PromptTemplate(
    template="Tell me a joke about {topic}"
)

parser = StrOutputParser()

response = parser.invoke(
    llm.invoke(
        prompt.invoke(
            {"topic": "Python"}
        )
    )
)
response

'Why do Python programmers prefer dark mode?\n\nBecause light attracts bugs!'

## **Runnables**

Runnables can be 
- executed
    - invoke(), 
    - batch() 
    - and stream()
- inspected,
- and composed

In [ ]:
runnables = [prompt, llm, parser]

#### Execute methods

In [ ]:
for runnable in runnables:
    print(f"{repr(runnable).split('(')[0]}")
    print(f"\tINVOKE: {repr(runnable.invoke)}")
    #print(f"\tBATCH: {repr(runnable.batch)}")
    #print(f"\tSTREAM: {repr(runnable.stream)}\n")

#### Inspect

In [ ]:
for runnable in runnables:
    print(f"{repr(runnable).split('(')[0]}")
    print(f"\tINPUT: {repr(runnable.get_input_schema())}")
    print(f"\tOUTPUT: {repr(runnable.get_output_schema())}")
    print(f"\tCONFIG: {repr(runnable.config_schema())}\n")

#### <mark>Config</mark>

In [ ]:
with collect_runs() as run_collection:
    result = llm.invoke(
        "Hello", 
        config={
            'run_name': 'demo_run', 
            'tags': ['demo', 'lcel'], 
            'metadata': {'lesson': 2}
        }
    )

run_collection.traced_runs

run_collection.traced_runs[0].dict()


#### <mark>Compose Runnables</mark>

In [ ]:
chain = RunnableSequence(prompt, llm, parser)

print(type(chain))

chain.invoke({"topic": "Python"})

In [ ]:
for chunk in chain.stream({"topic": "Python"}):
    print(chunk, end="", flush=True)

In [ ]:
chain.batch([
    {"topic": "Python"},
    {"topic": "Data"},
    {"topic": "Machine Learning"},
])

In [ ]:
chain.get_graph().print_ascii()

#### <mark>Turn any function into a runnable</mark>

In [ ]:
def my_function(x:int)->int:
    return 2*x

In [ ]:
my_runnable = RunnableLambda(my_function)
my_runnable.invoke(2)

#### Parallel Runnables

In [ ]:
parallel_chain = RunnableParallel(
    double=RunnableLambda(lambda x: x * 2),
    triple=RunnableLambda(lambda x: x * 3),
)

In [ ]:
parallel_chain.invoke(3)

In [ ]:
parallel_chain.get_graph().print_ascii()

## **LCEL**

In [ ]:
prompt

In [ ]:
llm

In [ ]:
parser

In [ ]:
chain = RunnableSequence(prompt, llm, parser)
chain

In [ ]:
prompt | llm | parser

In [ ]:
chain = prompt | llm | parser

In [ ]:
chain.invoke(
    {"topic": "computer"}
)